In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import urllib.request

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'



r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')
#print(soup.prettify())

In [3]:

table = soup.find('table', class_ = 'wikitable sortable')
#print(match.prettify())

header = table.tbody.tr.th.text
print(header)

Postcode


In [4]:

for table in soup.find_all('table', class_ = 'wikitable sortable'):
    header = table.tbody.tr.text
    print(header)
header = header.rstrip('\n')
header = header.lstrip('\n')
header = header.replace('\n', ',')
header_list = header.split(',')
header_list


Postcode
Borough
Neighbourhood



['Postcode', 'Borough', 'Neighbourhood']

In [5]:
entry_list = []
count = 0
for table in soup.find_all('table', class_ = 'wikitable sortable'):
    entry = table.tbody.text
    entry = entry.rstrip('\n')
    entry = entry.rstrip(' ')
    entry = entry.lstrip('\n')
    entry = entry.lstrip(' ')
    entry = entry.replace('\n', ',')
    
    entry_list.append(entry)
    entry_list = entry.split(',')
    print()

In [6]:
#clean it up a bit
for element in entry_list:
    try:
        entry_list.remove('')
    except ValueError:
        pass
#entry_list
del entry_list[0:3] # we already have these in header_list
entry_list[0:10]

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A']

In [7]:
import pandas as pd
import numpy as np

In [8]:
df_toronto = pd.DataFrame(data = np.array(entry_list).reshape(287,3), columns = header_list)
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:

df_toronto.replace('Not assigned', np.nan, inplace=True)
df_toronto = df_toronto.dropna(subset=['Borough'])
print(df_toronto.shape)
df_toronto.head(10)

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,NaN
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [13]:

df_toronto.fillna(0, inplace=True)
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,0
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [14]:
mask = df_toronto['Neighbourhood'] == 0
df_toronto.loc[mask, 'Neighbourhood'] = df_toronto['Borough']
df_toronto['Neighbourhood'] = df_toronto['Neighbourhood'].mask(mask, df_toronto['Borough'])
print(df_toronto.shape)
df_toronto.head(10)

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [15]:
df_toronto_grouped = df_toronto.groupby(['Postcode', 'Borough']).agg({'Neighbourhood':lambda x: ', '.join(x)}).reset_index()
df_toronto_grouped.head(10) #only shows Scarborough because dataframe sorted by postcode

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
#If you want to see more of the dataframe to verify it is consistent with the desired solution on the coursera page
df_toronto_grouped

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [17]:
url2 = 'http://cocl.us/Geospatial_data'
df_ll = pd.read_csv(url2)
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_ll.rename({'Postal Code':'Postcode'}, axis=1, inplace=True)
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_toronto_ll = pd.merge(df_toronto_grouped, df_ll, on='Postcode')
df_toronto_ll.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
from geopy.geocoders import Nominatim
#import folium
import json
import requests
from pandas.io.json import json_normalize

In [22]:
toronto_east = df_toronto_ll[df_toronto_ll['Borough'] == 'East Toronto'].reset_index(drop=True)
toronto_west = df_toronto_ll[df_toronto_ll['Borough'] == 'West Toronto'].reset_index(drop=True)
toronto_DT = df_toronto_ll[df_toronto_ll['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
frames = [toronto_east, toronto_west, toronto_DT]
toronto_hoods = pd.concat(frames).set_index('Postcode').reset_index()
toronto_hoods
toronto_hoods

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
5,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
6,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
7,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
8,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
9,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325


In [23]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [25]:
CLIENT_ID = 'BQXM0NHEJLDDRQE5VX1U10E1VOXRVXRMH1I2N30LZJDVSY5W' # your Foursquare ID
CLIENT_SECRET = 'R22G1UF3WEP2F0YYYJ4MY2MSN4ODZRLDRKJKAU10WGFMYIJW' # your Foursquare Secret
VERSION = '20190403' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BQXM0NHEJLDDRQE5VX1U10E1VOXRVXRMH1I2N30LZJDVSY5W
CLIENT_SECRET:R22G1UF3WEP2F0YYYJ4MY2MSN4ODZRLDRKJKAU10WGFMYIJW


In [26]:
toronto_hoods.loc[1, 'Neighbourhood']

'The Danforth West, Riverdale'

In [27]:
hood_lat = toronto_hoods.loc[1, 'Latitude'] # neighborhood latitude value
hood_lon = toronto_hoods.loc[1, 'Longitude'] # neighborhood longitude value

hood_name = toronto_hoods.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(hood_name, 
                                                               hood_lat, 
                                                               hood_lon))

Latitude and longitude values of The Danforth West, Riverdale are 43.6795571, -79.352188.


In [28]:
# Create GET request url
radius = 500
LIMIT = 50
url3 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hood_lat, hood_lon, VERSION, radius, LIMIT)
url3

'https://api.foursquare.com/v2/venues/explore?client_id=BQXM0NHEJLDDRQE5VX1U10E1VOXRVXRMH1I2N30LZJDVSY5W&client_secret=R22G1UF3WEP2F0YYYJ4MY2MSN4ODZRLDRKJKAU10WGFMYIJW&ll=43.6795571,-79.352188&v=20190403&radius=500&limit=50'

In [30]:
results = requests.get(url3).json()

#function that extracts venue category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:

# Clean json and convert to pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pantheon,Greek Restaurant,43.677621,-79.351434
1,MenEssentials,Cosmetics Shop,43.677820,-79.351265
2,Cafe Fiorentina,Italian Restaurant,43.677743,-79.350115
3,Mezes,Greek Restaurant,43.677962,-79.350196
4,Dolce Gelato,Ice Cream Shop,43.677773,-79.351187


In [32]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:

toronto_venues = getNearbyVenues(names=toronto_hoods['Neighbourhood'],
                                   latitudes=toronto_hoods['Latitude'],
                                   longitudes=toronto_hoods['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(1092, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [36]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,50,50,50,50,50,50
"Chinatown, Grange Park, Kensington Market",50,50,50,50,50,50
Christie,17,17,17,17,17,17


In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 205 uniques categories.


In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020,0.000000,0.00,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020,0.000000,0.00,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.062500,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000000,0.000000
6,Central Bay Street,0.020000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.020,0.000000,0.00,0.020000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.060,0.000000,0.06,0.020000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000000,0.000000
9,Church and Wellesley,0.020000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.02,0.000000,0.000000,0.00,0.000,0.000000,0.02,0.000000,0.020000


In [40]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Steakhouse,Asian Restaurant,Gastropub,Breakfast Spot,Pizza Place,American Restaurant,Hotel,Coffee Shop,Opera House
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Café,Diner,Basketball Stadium
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Pet Store,Climbing Gym,Stadium,Bakery,Intersection,Italian Restaurant,Convenience Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Park,Pizza Place,Comic Shop,Burrito Place,Restaurant,Brewery,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Boutique,Bar
